In [1]:
# %pip install tensorflow_probability 

# CNN a GM


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt


datos


In [ ]:
# 02_main.ipynb
from grav_lens import get_datasets
from grav_lens.models import test_model_image

import os 

home_data = os.path.join("..","data")
train_dataset, val_dataset, test_dataset = get_datasets(data_index='1', max_files=1500, 
    home=home_data)

import matplotlib.pyplot as plt
import numpy as np


def plot_history(history):
	# Crear una figura
	plt.figure(figsize=(12, 8))

	line_styles = ['-', '--', '-.', ':']
	for i, key in enumerate(history.history.keys()):
		plt.plot(history.history[key], label=key, linestyle=line_styles[i % len(line_styles)])  # Estilo de línea diferente

	plt.yscale('log')
	# Configurar los detalles de la gráfica
	plt.title('Evolución de las Pérdidas y Métricas durante el Entrenamiento')
	plt.xlabel('Épocas')
	plt.ylabel('Valor')
	plt.legend()
	plt.grid(True)
	plt.show()


Modelo Gaussiano
Toma los parametros y construye la imagen

In [91]:
class GaussianImageLayer(tf.keras.layers.Layer):
    def __init__(self, img_size=64, Ng=10, sigma_penalty_weight=0.01, c_pi_penalty_weight=0.01, **kwargs):
        super(GaussianImageLayer, self).__init__(**kwargs) 
        self.img_size = img_size  # Tamaño de la imagen generada
        self.Ng = Ng  # Número de Gaussianas
        self.sigma_penalty_weight = sigma_penalty_weight  # Peso de la penalización sobre sigma
        self.c_pi_penalty_weight = c_pi_penalty_weight  # Peso de la penalización sobre c_pi

    def call(self, flat_tensor):
        """Construcción de la imagen con múltiples Gaussianas a partir del tensor plano."""
        batch_size = tf.shape(flat_tensor)[0]
        
        # Ahora Ng es un parámetro fijo pasado al constructor
        Ng = self.Ng

        # Extraer c_pi, mu_x, mu_y, sigma_x, sigma_y
        c_pi = flat_tensor[:, :Ng]  # Coeficientes de mezcla
        mu_x = flat_tensor[:, Ng:2*Ng]  # Media en x
        mu_y = flat_tensor[:, 2*Ng:3*Ng]  # Media en y
        sigma_x = tf.abs(flat_tensor[:, 3*Ng:4*Ng]) + 0.1  # Desviación estándar en x (mínimo de 0.1)
        sigma_y = tf.abs(flat_tensor[:, 4*Ng:]) + 0.1  # Desviación estándar en y (mínimo de 0.1)

        # Añadir penalización de sigma en la loss (L2 regularization para mantener los valores pequeños)
        self.add_loss(self.sigma_penalty_weight * tf.reduce_sum(tf.square(sigma_x) + tf.square(sigma_y)))

        # Añadir penalización de c_pi en la loss (L2 regularization para mantener los valores de c_pi cercanos a 0)
        self.add_loss(self.c_pi_penalty_weight * tf.reduce_sum(tf.square(c_pi)))

        # Normalizar mu_x y mu_y al rango de la grilla [-1, 1]
        mu_x = mu_x * 2 - 1  # Transformar de [0, 1] a [-1, 1]
        mu_y = mu_y * 2 - 1  # Transformar de [0, 1] a [-1, 1]

        # Crear una grilla de coordenadas (img_size x img_size)
        x = tf.linspace(-1.0, 1.0, self.img_size)
        y = tf.linspace(-1.0, 1.0, self.img_size)
        X, Y = tf.meshgrid(x, y)  # Crear grilla de coordenadas
        coords = tf.stack([X, Y], axis=-1)  # Combinar coordenadas (img_size, img_size, 2)

        # Expandir dimensiones para batch y Ng
        coords = tf.reshape(coords, (1, self.img_size, self.img_size, 2))  # (1, img_size, img_size, 2)
        coords = tf.tile(coords, [batch_size, 1, 1, 1])  # (batch_size, img_size, img_size, 2)

        # Inicializar la imagen final en ceros
        final_image = tf.zeros((batch_size, self.img_size, self.img_size), dtype=tf.float32)

        # Iterar sobre el número de Gaussianas (Ng)
        for i in range(Ng):
            # Obtener las medias y desviaciones para la Gaussiana i
            mu = tf.stack([mu_x[:, i], mu_y[:, i]], axis=-1)  # (batch_size, 2)
            sigma = tf.stack([sigma_x[:, i], sigma_y[:, i]], axis=-1)  # (batch_size, 2)

            # Expandir las dimensiones de mu y sigma para que coincidan con las coordenadas de la imagen
            mu = tf.expand_dims(mu, axis=1)  # (batch_size, 1, 2)
            mu = tf.expand_dims(mu, axis=1)  # (batch_size, 1, 1, 2)
            sigma = tf.expand_dims(sigma, axis=1)  # (batch_size, 1, 2)
            sigma = tf.expand_dims(sigma, axis=1)  # (batch_size, 1, 1, 2)

            # Crear una distribución normal multivariada para cada batch y Gaussiana
            mvn = tfp.distributions.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
            
            # Calcular las probabilidades en las coordenadas de la imagen
            gaussians = mvn.prob(coords)  # (batch_size, img_size, img_size)
            
            # Ponderar cada Gaussiana por c_pi (ajustar dimensiones para que coincidan)
            gaussians_weighted = gaussians * tf.expand_dims(tf.expand_dims(c_pi[:, i], axis=-1), axis=-1)

            # Sumar la Gaussiana ponderada a la imagen final
            final_image += gaussians_weighted

        # Expandir las dimensiones de la imagen final para que sea (batch_size, img_size, img_size, 1)
        return tf.expand_dims(final_image, axis=-1)

    def compute_output_shape(self, input_shape):
        # Definir la forma de salida, que será (batch_size, img_size, img_size, 1)
        return (input_shape[0], self.img_size, self.img_size, 1)


# Construccion del modelo

In [93]:
import tensorflow as tf
from tensorflow.keras import layers, models
from grav_lens import metrics

def create_cnn_model(hyperparams):
	"""
	Función para construir un modelo CNN secuencial con hiperparámetros ajustables a través de un diccionario.
	
	Parámetros:
	- hyperparams: Diccionario con los hiperparámetros del modelo.
	
	Retorna:
	- Un modelo CNN construido con los hiperparámetros especificados.
	"""
	
	img_size = hyperparams['img_size']
	Ng = hyperparams['Ng']
	num_conv_layers = hyperparams['num_conv_layers']
	conv_filters = hyperparams['conv_filters']
	kernel_size = hyperparams['kernel_size']
	dense_units = hyperparams['dense_units']
	num_dense_layers = hyperparams['num_dense_layers']
	activation = hyperparams['activation']
	output_activation = hyperparams['output_activation']
	learning_rate = hyperparams['learning_rate']
	

	input_layer = tf.keras.layers.Input(shape=(128, 128, 3))

	initializer = tf.keras.initializers.GlorotNormal()
	x = layers.Conv2D(conv_filters, kernel_size, 
		activation=activation, padding='same', kernel_initializer=initializer)(input_layer)

	# Capas convolucionales hidden 
	for _ in range(num_conv_layers):
		x = layers.Conv2D(conv_filters*4, kernel_size, 
			activation=activation, padding='same', kernel_initializer=initializer)(x)

		x = layers.MaxPooling2D(pool_size=(2, 2))(x)
	
	
	# Aplanar la salida de las convolucionales
	x = layers.Flatten()(x)

	# Capas densas ocultas
	for _ in range(num_dense_layers):
		x = layers.Dense(dense_units, activation=activation, kernel_initializer=initializer)(x)
	
	# Capa de salida para generar el vector plano
	output_units = 5 * Ng  # 5 parámetros por cada Gaussiana
	x = layers.Dense(output_units, activation=output_activation, kernel_initializer=initializer)(x)

	out_img = GaussianImageLayer(img_size=128, Ng=Ng, name='out_img')(x)
	
	model = tf.keras.models.Model(inputs=input_layer, 
		outputs=[out_img, out_img] )

	optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


	model.compile(optimizer=optimizer, 
				loss= ['mse', metrics.WMAPE, 'mae'],
				metrics={'out_img': [metrics.DICEE, metrics.WMAPE]})


	return model


Model: "functional_73"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_80 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_81 (Conv2D)              │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_68 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_82 (Conv2D)              │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_69 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_83 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_70 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_84 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_71 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 50)             │         6,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_img (GaussianImageLayer)    │ (None, 128, 128, 1)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,122 (1.23 MB)

 Trainable params: 323,122 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

# Crear el modelo

In [ ]:

# Hiperparámetros como un diccionario
hyperparams = {
	'img_size': 128,               # Tamaño de la imagen de entrada
	'Ng': 10,                     # Número de Gaussianas (esto afecta el tamaño de salida)
	'num_conv_layers': 4,         # Número de capas convolucionales
	'conv_filters': 32,           # Número de filtros en cada capa convolucional
	'kernel_size': (3, 3),        # Tamaño del kernel
	'dense_units': 128,           # Número de unidades en las capas densas
	'num_dense_layers': 2,        # Número de capas densas
	'activation': 'tanh',         # Función de activación en las capas
	'output_activation': 'sigmoid',    # Sin activación final (los valores pueden ser negativos o positivos)
	'learning_rate': 0.0001        # Tasa de aprendizaje
}

# Crear el modelo utilizando el diccionario de hiperparámetros
cnn_model = create_cnn_model(hyperparams)

## Training

In [101]:

history = cnn_model.fit(train_dataset, validation_data=val_dataset, epochs=30, verbose=True)


Using data folder: ..\data\1
Epoch 1/30
     17/Unknown 27s 1s/step - loss: 0.2551 - out_img_dicee: 0.5567 - out_img_wmape: 45.5516

C:\Users\fbien.DESKTOP-6FMEAR7\Documents\python\lib\contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 0.2540 - out_img_dicee: 0.5550 - out_img_wmape: 45.4194 - val_loss: 0.2364 - val_out_img_dicee: 0.4707 - val_out_img_wmape: 41.1738
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - loss: 0.2295 - out_img_dicee: 0.5621 - out_img_wmape: 42.4827 - val_loss: 0.2160 - val_out_img_dicee: 0.4760 - val_out_img_wmape: 37.8479
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 0.2104 - out_img_dicee: 0.5675 - out_img_wmape: 39.0988 - val_loss: 0.1994 - val_out_img_dicee: 0.4814 - val_out_img_wmape: 34.9537
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 0.1945 - out_img_dicee: 0.5748 - out_img_wmape: 36.8791 - val_loss: 0.1857 - val_out_img_dicee: 0.4850 - val_out_img_wmape: 32.4138
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 0.1814 - out_img_dicee: 0.5786 - out_img_wmape: 33.2199 - val_loss: 0.1741 - val_out_img_dicee: 0.4883 - val_out_img_wmape: 30.1670
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 0.1

In [ ]:
plot_history(history)

In [ ]:
test_model_image(cnn_model, test_dataset, outs=2)